# Most basic time decomposition simulation
We assume a flare with fixed physical parameters and two basis light curves.

The basis light curves are fractional browninan motion with $H = 0.5$ and $H = 0.955$.

The instrument response matrix is ignored for now.

The background is randomly distributed around a mean value,
with $\sigma = 4\mu$,
such that the nonthermal component appears washed out above 50 keV.

The physical parameters are as follows

**Thermal, isothermal**
- $\text{T} = 20$ MK
- $\text{EM} = 1 \times 10^{49} \text{ cm}^{-3}$

**Nonthermal, thick target**
- $\varphi_e = 6\times 10^{35}$ electron/second
- $\delta = 4$
- $\text{E}_c = 25$ keV

In [ ]:
import os
import copy

from astropy import units as u
from astropy import visualization as viz
from matplotlib import pyplot as plt
import numpy as np

import scipy.stats as st
from yaff import common_models as cm
from yaff.fitting import Parameter
from yaff import plotting

from tedec import fractional_brownian_motion as fbm
from tedec import decomp

%matplotlib qt
plt.style.use(os.getenv("MPL_INTERACTIVE_STYLE"))

## Generate some basis light curves to use later

In [ ]:
seed = 132457
np.random.seed(seed)
time_bin = 0.1
integration = 30
steps = int(integration / time_bin)
thermal_basis = fbm.make_timeseries(num=steps, hurst=0.955)
nonthermal_basis = fbm.make_timeseries(num=steps, hurst=0.5)

In [ ]:
def rebin_clumps(histogram, clump_size):
    ret = np.zeros(histogram.size // clump_size)
    for i in range(0, histogram.size, clump_size):
        ret[i // clump_size] = histogram[i:i+clump_size].sum()
    return ret

# Bin down the time granularity
real_dt = 0.5
bin_down_factor = int(real_dt / time_bin)
thermal_basis = rebin_clumps(thermal_basis, bin_down_factor)
nonthermal_basis = rebin_clumps(nonthermal_basis, bin_down_factor)

In [ ]:
fig, ax = plt.subplots()
axx = ax.twinx()
t = np.arange(thermal_basis.size + 1)
ax.stairs(thermal_basis, t, color='red', label='thermal')
ax.legend(loc='upper left')
ax.set(ylabel='thermal magnitude')

axx.stairs(nonthermal_basis, t, color='black', label='nonthermal')
axx.legend(loc='lower right')
axx.set(ylabel='nonthermal magnitude')

for a in (ax, axx):
    a.spines['left'].set_color('red')
ax.yaxis.label.set_color('red')
ax.tick_params(axis='y', colors='red', which='both')

plt.show()

In [ ]:
def normalize(s):
    # s -= s.min()
    # s += 4
    return np.nan_to_num(s / s.sum())

norm_th = normalize(thermal_basis)
norm_nth = normalize(nonthermal_basis)

fig, ax = plt.subplots()
ax.stairs(norm_th, t, label='thermal normalized', color='red')
ax.stairs(norm_nth, t, label='nonthermal normalized', color='black')
ax.legend()
plt.show()

## Take the basis lightcurves and use them to scale $\text{EM}$ and $\varphi_e$

In [ ]:
# Vary EM by factor of 3
th_scale = norm_th - norm_th.min()
th_scale /= th_scale.max()
th_scale *= 2
th_scale += 1

# Vary electron flux by max factor of 4
nth_scale = norm_nth - norm_nth.min()
nth_scale /= nth_scale.max()
nth_scale *= 3
nth_scale += 2

In [ ]:
thermal_physical_params = {
    'temperature': Parameter(20 << u.MK, True),
    'emission_measure': Parameter(1 << (1e49 * u.cm**-3), True)
}

nonthermal_physical_params = {
    'electron_flux': Parameter(6 << (1e35 * u.electron / u.s), True),
    'spectral_index': Parameter(4 << u.one, True),
    'cutoff_energy': Parameter(25 << u.keV, True)
}

energies = np.geomspace(3, 100, num=40) << u.keV

all_args = {
    'photon_energy_edges': energies.to_value(u.keV),
    'parameters': (thermal_physical_params | nonthermal_physical_params)
}

thermal_args = {
    'photon_energy_edges': energies.to_value(u.keV),
    'parameters': thermal_physical_params
}

nonthermal_args = {
    'photon_energy_edges': energies.to_value(u.keV),
    'parameters': nonthermal_physical_params
}

In [ ]:
def model(params: cm.ArgsT):
    return cm.thermal(params) + cm.thick_target(params)

In [ ]:
de = energies[1:] - energies[:-1]
area = 10 << u.cm**2

def flux_to_photons(func, args):
    flux_unit = (u.ph / u.cm**2 / u.s / u.keV)
    return ((func(args) << flux_unit) * de * (real_dt << u.s) * area).to(u.ph)


thermal_truth = list()
nonthermal_truth = list()
spectrogram = list()
for i in range(th_scale.size):
    # Update the EM and phi_e by the scale factor
    args = copy.deepcopy(all_args)
    args['parameters']['emission_measure'].value *= th_scale[i]
    args['parameters']['electron_flux'].value *= nth_scale[i]

    thermal_truth.append(flux_to_photons(cm.thermal, args))
    nonthermal_truth.append(flux_to_photons(cm.thick_target, args))
    spectrogram.append(flux_to_photons(model, args))

def strip_unit(a):
    return (a << u.ph).value.T

spectrogram = strip_unit(spectrogram)
thermal_truth = strip_unit(thermal_truth)
nonthermal_truth = strip_unit(nonthermal_truth)

In [ ]:
tests = np.arange(spectrogram.shape[0])

fig, ax = plt.subplots()
for test in tests:
    ax.stairs(spectrogram[test], real_dt * t)
ax.set(xlabel='time (s)', ylabel='photons incident')
plt.show()

In [ ]:
from matplotlib import colors as mcol

fig, ax = plt.subplots()
norm = mcol.LogNorm(vmin=spectrogram.min(), vmax=spectrogram.max())
ax.pcolormesh(t * real_dt, energies, spectrogram, norm=norm)
ax.set(xlabel='time', ylabel='energy')
plt.show()

In [ ]:
energy_mids = (energies[:-1] + np.diff(energies)/2).to_value(u.keV)
closest = lambda a, v: np.argmin(np.abs(a - v))

'''
Let's say we have a 5 uCurie Ba133 source on board.
For X-rays that's about 2e5 count/second.
There will be lines at 4 keV, 31 keV, and 81 keV
'''
count_rate = 600
num_time_bins = thermal_basis.size
noise = (count_rate * num_time_bins * real_dt) * np.ones(spectrogram.shape[0])
noise[closest(energy_mids, 4):closest(energy_mids, 6)] *= 100
noise[closest(energy_mids, 29):closest(energy_mids, 32)] *= 10
noise[closest(energy_mids, 80):closest(energy_mids, 84)] *= 5

rate = noise / num_time_bins
background = (st.poisson.rvs(np.tile(rate, (num_time_bins, 1))) << u.ct).T

In [ ]:
from matplotlib import colors as mcol

fig, ax = plt.subplots()
# ax.pcolormesh(t, energies, background)
norm = mcol.LogNorm()
ax.pcolormesh(t, energies.value, background.value, norm=norm)
ax.set(xlabel='time', ylabel='energy')
plt.show()

In [ ]:
# Add counting statistics & systematics onto the photon data
systematic = 0.05
data = st.norm.rvs(loc=spectrogram, scale=np.sqrt(spectrogram + (systematic * spectrogram)**2)) << u.ct

# Insert the background from the radioactive source
noisy_data = data + background
if noisy_data.min() < 0:
    raise ValueError("Can't have negative counts")

In [ ]:
from matplotlib import colors as mcol

fig, ax = plt.subplots()
norm = mcol.LogNorm(vmin=100, vmax=None)
pcm = ax.pcolormesh(t, energies, noisy_data.value, norm=norm)
fig.colorbar(pcm, label='photons')
ax.set(xlabel='time', ylabel='energy')
plt.show()

In [ ]:
reconstructed = spectrogram.sum(axis=1)
fig, ax = plt.subplots()

with viz.quantity_support():
    ax.stairs(reconstructed, energies, label='reconstructed data', linestyle='dashed', color='red')
    ax.stairs(noisy_data.sum(axis=1), energies, label='noisy, reconstructed data', linestyle='dashed', color='orange')
    ax.stairs(background.sum(axis=1), energies, label='background')

ax.legend()
ax.set(xscale='log', yscale='log', ylabel='photons', xlabel='energy keV')
plt.show()

In [ ]:
nearest = lambda a, v: np.argmin(np.abs(a - v))

thermal_index = nearest(energy_mids, 4)
nonthermal_index = nearest(energy_mids, 21)

dat = noisy_data.value

pack = decomp.DataPacket(
    data=dat,
    basis_timeseries=[
        dat[thermal_index],
        dat[nonthermal_index],
        dat[-3]
    ],
    constant_offset=False
)

systematic = 0.05
ret = decomp.bootstrap(
    pack,
    errors=np.sqrt(dat + (systematic * dat)**2),
    num_iter=1000
)

In [ ]:
th_mean = ret[:, 0, :].mean(axis=0) << u.ph
th_std = ret[:, 0, :].std(axis=0) << u.ph
nth_mean = ret[:, 1, :].mean(axis=0) << u.ph
nth_std = ret[:, 1, :].std(axis=0) << u.ph

# scale by # time bins (need to update)
bkg_part = ret[:, 2, :]# * th_scale.size
bkg_mean = bkg_part.mean(axis=0) << u.ph
bkg_std = bkg_part.std(axis=0) << u.ph

fig, ax = plt.subplots()

with viz.quantity_support():
    ax.stairs(thermal_truth.sum(axis=1), energies, label='true thermal spectrum')
    ax.stairs(nonthermal_truth.sum(axis=1), energies, label='true nonthermal spectrum')
    ax.stairs(background.sum(axis=1), energies, label='true background')
    
    num_sigma = 2
    plotting.stairs_with_error(energies, th_mean, num_sigma*th_std, ax=ax, label='decomposed thermal')
    plotting.stairs_with_error(energies, nth_mean, num_sigma*nth_std, ax=ax, label='decomposed nonthermal')
    plotting.stairs_with_error(energies, bkg_mean, num_sigma*bkg_std, ax=ax, label='bkg decom')

ax.legend()
ax.set(xscale='log', yscale='log', ylim=(1e3, None))
plt.show()

## Fit the individually decmoposed components

### Fit nonthermal decomposed data

In [ ]:
from yaff import fitting
from yaff import common_likelihoods

systematic = lambda s, c, a: np.sqrt(s**2 + ((c * a).value << s.unit)**2)

sys = 0.05
nth_data = fitting.DataPacket(
    counts=(nth_as_cts := nth_mean.to_value(u.ph) << u.ct),
    counts_error=systematic(nth_std, nth_as_cts, sys).to_value(u.ph) << u.ct,
    background_counts=(0 * nth_as_cts),
    background_counts_error=(0 * nth_as_cts),
    effective_exposure=(integration << u.s),
    count_energy_edges=energies,
    photon_energy_edges=energies,
    response_matrix=area * (np.eye(nth_as_cts.size) << (u.ct / u.ph))
)

nonthermal_priors = {
    'electron_flux': fitting.simple_bounds(0, 20),
    'spectral_index': fitting.simple_bounds(2.1, 10),
    'cutoff_energy': fitting.simple_bounds(5, 80),
}

rng = np.random.default_rng()
params = {
    k: fitting.Parameter(v.as_quantity() * rng.uniform(0.9, 1.1), frozen=False)
    for (k, v) in nonthermal_physical_params.items()
}

fit_range = (energy_mids > 4)
likelihood = common_likelihoods.chi_squared_factory(fit_range)

fr = fitting.BayesFitter(
    data=nth_data,
    model_function=cm.thick_target,
    parameters=params,
    log_priors=nonthermal_priors,
    log_likelihood=likelihood
)

In [ ]:
fr = fitting.levenberg_minimize(fr)
fr.parameters

In [ ]:
fr.run_emcee(
    emcee_constructor_kw={'nwalkers': 20},
    emcee_run_kw={'nsteps': 1000}
)


In [ ]:
from yaff import plotting as yap
yap.plot_parameter_chains(
    fr,
    names=fr.free_param_names,
    params=list(fr.free_parameters)
)
plt.show()

In [ ]:
samples = fr.generate_model_samples(100)
fig = plt.figure()
yap.plot_data_model(fr, model_samples=samples, fig=fig)
plt.show()

In [ ]:
import corner

burnin = (50 * fr.emcee_sampler.nwalkers)
corner_chain = fr.emcee_sampler.flatchain[burnin:]
param_names = fr.free_param_names

fig = plt.figure(figsize=(10, 8), layout="tight")
corner.corner(
    corner_chain,
    fig=fig,
    bins=20,
    labels=param_names,
    quantiles=(0.05, 0.5, 0.95),
    show_titles=True,
    truths=(
        nonthermal_physical_params['electron_flux'].value * nth_scale.mean(),
        nonthermal_physical_params['spectral_index'].value,
        nonthermal_physical_params['cutoff_energy'].value,
    ),
    # plot_contours=False,
    range=(
        (14, 20),
        (3.9, 4.4),
        (24, 30)
    ),
    truth_color='red'
)
fig.savefig('decomp nonthermal.png', dpi=300)
plt.show()

### Fit thermal decomposed data

In [ ]:
from yaff import fitting
from yaff import common_likelihoods

th_data = fitting.DataPacket(
    counts=(th_as_cts := th_mean.to_value(u.ph) << u.ct),
    counts_error=systematic(th_std, th_as_cts, sys).to_value(u.ph) << u.ct,
    background_counts=(0 * th_as_cts),
    background_counts_error=(0 * th_as_cts),
    effective_exposure=(integration << u.s),
    count_energy_edges=energies,
    photon_energy_edges=energies,
    response_matrix=area * (np.eye(nth_as_cts.size) << (u.ct / u.ph))
)

thermal_priors = {
    'temperature': fitting.simple_bounds(10, 40),
    'emission_measure': fitting.simple_bounds(1e-4, 1e4),
}

rng = np.random.default_rng()
params = {
    k: fitting.Parameter(v.as_quantity() * rng.uniform(0.9, 1.1), frozen=False)
    for (k, v) in thermal_physical_params.items()
}

likelihood = common_likelihoods.chi_squared_factory(
    restriction=(restriction := (energy_mids < 20))
)

fr = fitting.BayesFitter(
    data=th_data,
    model_function=cm.thermal,
    parameters=params,
    log_priors=thermal_priors,
    log_likelihood=likelihood
)

In [ ]:
fitting.levenberg_minimize(fr, restriction=restriction)

In [ ]:
fr.parameters

In [ ]:
fr.run_emcee(
    emcee_constructor_kw={'nwalkers': 20},
    emcee_run_kw={'nsteps': 1000}
)


In [ ]:
from yaff import plotting as yap
yap.plot_parameter_chains(
    fr,
    names=fr.free_param_names,
    params=list(fr.free_parameters)
)
plt.show()

In [ ]:
import corner

burnin = (100 * fr.emcee_sampler.nwalkers)
corner_chain = fr.emcee_sampler.flatchain[burnin:]
param_names = fr.free_param_names

fig = plt.figure(figsize=(10, 8), layout="tight")
corner.corner(
    corner_chain,
    fig=fig,
    bins=20,
    labels=param_names,
    quantiles=(0.05, 0.5, 0.95),
    show_titles=True,
    truths=(
        thermal_physical_params['temperature'].value,
        thermal_physical_params['emission_measure'].value * th_scale.mean(),
    ),
    truth_color='red'
)
plt.savefig('decomp thermal.png', dpi=300)
plt.show()

In [ ]:
samples = fr.generate_model_samples(num=100)
fig = plt.figure()
yap.plot_data_model(fr, model_samples=samples, fig=fig)
plt.show()

## Do a traditional two-model fit

In [ ]:
dp = fitting.DataPacket(
    counts=(cts := noisy_data.sum(axis=1).value) << u.ct,
    counts_error=np.sqrt(cts + (sys * cts)**2) << u.ct,
    background_counts=(bg := background.sum(axis=1).value) << u.ct,
    background_counts_error=np.sqrt(bg + (sys * bg)**2) << u.ct,
    effective_exposure=(integration << u.s),
    count_energy_edges=energies,
    photon_energy_edges=energies,
    response_matrix=area * (np.eye(cts.size) << (u.ct / u.ph))
)

In [ ]:
priors = thermal_priors | nonthermal_priors
params = {
    k: fitting.Parameter(v.as_quantity() * rng.uniform(0.9, 1.1), frozen=False)
    for (k, v) in (thermal_physical_params | nonthermal_physical_params).items()
}

def model(args):
    return cm.thermal(args) + cm.thick_target(args)

likelihood = common_likelihoods.chi_squared_factory(restriction := energy_mids < 70)

fr = fitting.BayesFitter(
    data=dp,
    model_function=model,
    parameters=params,
    log_priors=priors,
    log_likelihood=likelihood
)

In [ ]:
fr.parameters

In [ ]:
fitting.levenberg_minimize(fr, restriction)

In [ ]:
fr.parameters

In [ ]:
fr.run_emcee(
    emcee_constructor_kw={'nwalkers': 20},
    emcee_run_kw={'nsteps': 1000}
)


In [ ]:
from yaff import plotting as yap
yap.plot_parameter_chains(
    fr,
    names=fr.free_param_names,
    params=list(fr.free_parameters)
)
plt.show()

In [ ]:
fr.parameters

In [ ]:
import corner

burnin = (50 * fr.emcee_sampler.nwalkers)
corner_chain = fr.emcee_sampler.flatchain[burnin:]
param_names = fr.free_param_names

fig = plt.figure(figsize=(20, 20), layout="tight")
corner.corner(
    corner_chain,
    fig=fig,
    bins=20,
    labels=param_names,
    quantiles=(0.05, 0.5, 0.95),
    show_titles=True,
    truths=(
        thermal_physical_params['temperature'].value,
        thermal_physical_params['emission_measure'].value * th_scale.mean(),
        nonthermal_physical_params['electron_flux'].value * nth_scale.mean(),
        nonthermal_physical_params['spectral_index'].value,
        nonthermal_physical_params['cutoff_energy'].value,
    ),
    truth_color='red'
)

plt.savefig('traditional.png', dpi=300)
plt.show()

In [ ]:
samples = fr.generate_model_samples(num=100)
fig = plt.figure()
yap.plot_data_model(fr, model_samples=samples, fig=fig)
plt.show()